In [1]:
import re
import time
import json
import requests
import pandas as pd
from transfermarket.market import Market
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [ ]:
# Use transferMarkt API to get Premier League Teams ID's to use in URL
tm = Market()
comp = tm.get_competitions()
clubs = tm.get_teams('GB1')
promoted_clubs = tm.get_teams('GB2')
team_ids = []
player_ids = {}
player_list = []
relegated_ids = [1132, 1031, 350]
promoted_ids = [677, 1003, 180]
for i in range(0, len(clubs)):
    if clubs[i].id not in relegated_ids:
        team_ids.append(clubs[i].id)
for i in range(0, len(promoted_clubs)):
    if promoted_clubs[i].id in promoted_ids:
        team_ids.append(promoted_clubs[i].id)
for i in range(0, len(team_ids)):
    player_list.append(tm.get_players(team_ids[i]))
    for j in range(0, len(player_list[i])):
        player_ids[player_list[i][j].name] = player_list[i][j].id
# save all player id and player names into a text file
with open("id_data.json", "w", encoding= 'utf-8') as f:
  json.dump(player_ids, f)

In [ ]:
#Extract the Age for all players based on Wage_Data.xlsx based on previous json output
diff_names=[]

#define headers for web scraping
headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
}
with open("id_data1.json", "r", encoding = 'utf-8') as f:
  player_ids = json.load(f)
player_wages_df = pd.read_excel("Wage Data.xlsx")

#Build the URL and request data
url = "https://r.jina.ai//www.transfermarkt.co.uk/fulham-fc/kader/verein/180/saison_id/2023/plus/1"
response = requests.get(url=url, headers=headers)
webpage_text = response.text
for i in range (0, len(player_wages_df['Player'])):
    player_name = str(player_wages_df['Player'][i])
    if player_name in webpage_text:
      age = re.findall(f"{player_name}[\s\S]+?\| .* \((\d+)\)", str(webpage_text))[0]
      player_wages_df['Age'][i] = int(age)
    else:
         continue
    try:
            id = player_ids[player_name]
    except Exception as ex:
            diff_names.append(player_name)
            continue
#player_wages_df.to_excel("Wage Data.xlsx")

In [ ]:
#Extract Injury History for each player based on Player names from URL

#define headers for web scraping
headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
}
player_wages_df = pd.read_excel("Wage Data.xlsx")
diff_names = []
player = []
inj_data = []
for i in range (0, len(player_wages_df['Player'])):
    if(player_wages_df['Team'][i] == 'AFC Bouremouth'):
        player_name = str(player_wages_df['Player'][i])
        try:
            id = player_ids[player_name]
        except Exception as ex:
            diff_names.append(player_name)
            continue
        url = f"https://r.jina.ai//www.transfermarkt.co.uk/{player_name}/verletzungen/spieler/{id}"
        response = requests.get(url=url, headers=headers)
        webpage_text = response.text
        extract = re.findall("\| (\d+/\d+) \| ([\w ]+) \| ([\w, ]+) \| ([\w, ]+) \| ([\w, ]+) \|", str(webpage_text))
        for i in range(0, len(extract)):
            inj_dict = {'Name': player_name, 'Season': extract[i][0], 'Injury': extract[i][1], 'From date': extract[i][2], 'Till date': extract[i][3], 'Days missed': extract[i][4]}
            inj_data.append(inj_dict)
        if(response.status_code != 200):
            print(response.status_code)
        time.sleep(3)
inj_df = pd.DataFrame(inj_data)
#inj_df.to_csv("inj_data.csv", mode='a+')

In [ ]:
#ISCO Analytics - create the ISCO combined dataset based on wage and injury data

player_wages_df = pd.read_excel("Wage Data.xlsx")
inj_df = pd.read_csv("inj_data.csv")
inj_df['Days missed'] = inj_df['Days missed'].str.replace(' days', '').astype(int)

total_days_injured_per_player = inj_df.groupby('Name')['Days missed'].sum().reset_index()
total_days_injured_per_player.columns = ['Player', 'Total Days Injured']

# Calculate injury frequency for each player (number of injuries)
injuries_per_player = inj_df.groupby('Name').size().reset_index()
injuries_per_player.columns = ['Player', 'Total Injury Count']

# Merge the wage data with the total days injured data
merged_df = pd.merge(player_wages_df, total_days_injured_per_player, left_on='Player', right_on='Player', how='left')
merged_df = pd.merge(merged_df, injuries_per_player, left_on='Player', right_on='Player', how='left')
merged_df['Total Days Injured'].fillna(0, inplace=True)
merged_df['Total Injury Count'].fillna(0, inplace=True)

# Convert 'Annual Salary' and 'Weekly Wage' to numerical values (remove currency symbols and commas)
merged_df['Annual Salary'] = merged_df['Annual Salary'].replace('[\£,]', '', regex=True).astype(float)
merged_df['Weekly Wage'] = merged_df['Weekly Wage'].replace('[\£,]', '', regex=True).astype(float)

# Calculate the average days missed per season for each player
merged_df['Days Injured per Season'] = (merged_df['Total Days Injured'] / (merged_df['Age'] - 17)).round(2)
merged_df['Days Missed per Injury'] = (merged_df['Total Days Injured'] / merged_df['Total Injury Count']).round(2)
merged_df['Days Missed per Injury'].fillna(0, inplace=True)
merged_df['Days Injured per Season'].fillna(0, inplace=True)
merged_df['Injury Risk Score'] = ((merged_df['Age'] * 0.1) + ((merged_df['Total Injury Count'] / merged_df['Age']) * 0.4) + merged_df['Days Injured per Season'] * 0.25 + merged_df['Days Missed per Injury'] * 0.25)

# Separate players with zero injuries and classify them as 'No Risk'
no_risk_players = merged_df[merged_df['Total Injury Count'] == 0].copy()
no_risk_players['Cluster Label'] = 'No Risk'

# Filter out players with zero injuries
cluster_df = merged_df[merged_df['Total Injury Count'] > 0].copy()

# Normalize the features
scaler = MinMaxScaler()
features = ['Injury Risk Score', 'Annual Salary']
scaled_features = scaler.fit_transform(cluster_df[features])

# Apply K-means clustering
kmeans = KMeans(n_clusters=6, random_state=0).fit(scaled_features)
cluster_df['Cluster'] = kmeans.labels_

# Manually assign labels based on the cluster characteristics
cluster_labels = {0: 'MediumRisk-HighWage', 1: 'HighRisk-LowWage', 2: 'VeryHighRisk-LowWage', 3: 'LowRisk-LowWage', 4: 'LowRisk-HighWage', 5: 'MediumRisk-LowWage'}
cluster_df['Cluster Label'] = cluster_df['Cluster'].map(cluster_labels)

# Get the cluster centers and transform back to original scale
cluster_centers = kmeans.cluster_centers_
original_centers = scaler.inverse_transform(cluster_centers)

# Get the boundaries for each cluster
boundaries = {}
for label in cluster_df['Cluster'].unique():
    cluster_data = cluster_df[cluster_df['Cluster'] == label]
    min_values = cluster_data[features].min()
    max_values = cluster_data[features].max()
    boundaries[label] = {
        'Cluster Label': cluster_labels[label],
        'Min Values': min_values,
        'Max Values': max_values
    }

# Combine the 'No Risk' players with the clustered players
merged_df = pd.concat([no_risk_players, cluster_df], ignore_index=True)

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='Annual Salary', y='Injury Risk Score', hue='Cluster Label', palette=sns.color_palette("Set2", 7))
plt.title('Clusters of Players Based on Injury Proneness')
plt.xlabel('Annual Salary')
plt.ylabel('Injury Risk Score')
plt.legend(title='Cluster Label', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

# Remove duplicate rows
merged_df.drop_duplicates(inplace=True)

merged_df.to_csv("ISCO.csv")


In [ ]:
# Function to calculate percentile rank
def calculate_percentile(series):
    return series.rank(pct=True) * 100

#Calculate percentage change proportional to injury risk score

isco_df = pd.read_csv("ISCO.csv")

#Max wage cut is 30% and min wage cut is 5%
maxcut = 30.0
mincut = 5.0
max_risk = isco_df['Injury Risk Score'].max()
min_risk = isco_df[isco_df['Injury Risk Score'] > mincut]['Injury Risk Score'].min()
isco_df['percent_change'] = 0.0
isco_df['FitWage'] = isco_df['Weekly Wage']
isco_df['InjuredWage'] = isco_df['Weekly Wage']
for i in range(0, len(isco_df['Injury Risk Score'])):
    if(isco_df['Injury Risk Score'][i]>mincut):
        percent_change = (maxcut - ((maxcut - mincut) * ((max_risk - min_risk - isco_df['Injury Risk Score'][i]) / (max_risk - min_risk)))).round(2)
        isco_df.at[i, 'percent_change'] = percent_change
        if "LowWage" in isco_df.at[i, 'Cluster Label']:
            isco_df.at[i, 'FitWage'] = isco_df['Weekly Wage'][i] + (isco_df['Weekly Wage'][i] * percent_change / 200).astype(int)
        else:
            isco_df.at[i, 'FitWage'] = isco_df['Weekly Wage'][i]
        isco_df.at[i, 'InjuredWage'] = isco_df['Weekly Wage'][i] - (isco_df['Weekly Wage'][i] * percent_change / 100).astype(int)
    else:
        isco_df.at[i, 'percent_change'] = 0
        isco_df.at[i, 'FitWage'] = isco_df['Weekly Wage'][i]
        isco_df.at[i, 'InjuredWage'] = isco_df['Weekly Wage'][i]

#Recommendation sentence based on Classification

    match isco_df.at[i, 'Cluster Label']:
        case "MediumRisk-HighWage":
            isco_df.at[i, 'Recommendation'] = "High Potential (Careful Management): Maximise value with strategic management"
        case "HighRisk-LowWage":
            isco_df.at[i, 'Recommendation'] = "Depth Option: Consider for backup option, injury risk a major factor."
        case "VeryHighRisk-LowWage":
            isco_df.at[i, 'Recommendation'] = "High-Risk Gamble: Potential upside, but cautious approach needed."
        case "LowRisk-LowWage":
            isco_df.at[i, 'Recommendation'] = "Reliable Depth: Solid contributors, good value."
        case "LowRisk-HighWage":
            isco_df.at[i, 'Recommendation'] = "Core Star: Build your team around them."
        case "MediumRisk-LowWage":
            isco_df.at[i, 'Recommendation'] = "Value Rotation: Good balance, valuable backups."
        case "No Risk":
            isco_df.at[i, 'Recommendation'] = "Dream Player: Minimal risk of injury in the future."

# Calculate percentile rank for 'Injury Risk Score' within each 'POS' and 'Team'
isco_df['Injury Risk Percentile_POS'] = 100 - isco_df.groupby(['POS'])['Injury Risk Score'].transform(calculate_percentile).round(2)
isco_df['Injury Risk Percentile_Team'] = 100 - isco_df.groupby(['Team'])['Injury Risk Score'].transform(calculate_percentile).round(2)

# Calculate percentile rank for 'Annual Salary' within each 'POS' and 'Team'
isco_df['Salary Percentile_POS'] = isco_df.groupby(['POS'])['Annual Salary'].transform(calculate_percentile).round(2)
isco_df['Salary Percentile_Team'] = isco_df.groupby(['Team'])['Annual Salary'].transform(calculate_percentile).round(2)

#Create final CSV data file
isco_df.to_csv("ISCO.csv")